# Exercicio PySpark

## Etapa 1

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
from pyspark.sql.functions import col, rand, when, udf, expr
import random
from pyspark.sql.types import StringType, IntegerType

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Exercicio Intro") \
    .getOrCreate()

24/07/29 00:51:33 WARN Utils: Your hostname, rafael resolves to a loopback address: 127.0.1.1; using 192.168.0.130 instead (on interface enp7s0)
24/07/29 00:51:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/29 00:51:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Etapa 2

In [2]:
schema = "Nomes STRING"
df_nomes = spark.read.csv("nomes_aleatorios.txt", header = False, schema = schema)
df_nomes.printSchema()
df_nomes.show(10)

root
 |-- Nomes: string (nullable = true)

+-----------------+
|            Nomes|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



## Etapa 3

In [3]:
df_nomes = df_nomes.withColumn("Escolaridade", when(rand() <= 1/3, "Fundamental").when(rand() <= 2/3, "Medio").otherwise("Superior"))
df_nomes.show(10)

+-----------------+------------+
|            Nomes|Escolaridade|
+-----------------+------------+
|   Frances Bennet| Fundamental|
|    Jamie Russell|       Medio|
|   Edward Kistler|    Superior|
|    Sheila Maurer| Fundamental|
| Donald Golightly| Fundamental|
|       David Gray|    Superior|
|      Joy Bennett|    Superior|
|      Paul Kriese| Fundamental|
|Berniece Ornellas|       Medio|
|    Brian Farrell|       Medio|
+-----------------+------------+
only showing top 10 rows



## Etapa 4

In [4]:
paises_america_do_sul = ["Brasil", "Argentina", "Uruguai", "Paraguai", "Chile", "Peru", "Colombia", "Venezuela", "Equador", "Bolivia", "Guiana", "Suriname", "Guiana Francesa"]

@udf(returnType=StringType())
def pais_aleatorio():
    return random.choice(paises_america_do_sul)

df_nomes = df_nomes.withColumn("Pais", pais_aleatorio())
df_nomes.show(10)

+-----------------+------------+---------+
|            Nomes|Escolaridade|     Pais|
+-----------------+------------+---------+
|   Frances Bennet| Fundamental| Suriname|
|    Jamie Russell|       Medio|  Equador|
|   Edward Kistler|    Superior| Paraguai|
|    Sheila Maurer| Fundamental|  Uruguai|
| Donald Golightly| Fundamental|Venezuela|
|       David Gray|    Superior| Colombia|
|      Joy Bennett|    Superior|  Uruguai|
|      Paul Kriese| Fundamental|   Brasil|
|Berniece Ornellas|       Medio| Suriname|
|    Brian Farrell|       Medio|Venezuela|
+-----------------+------------+---------+
only showing top 10 rows



## Etapa 5

In [5]:
anos_possiveis = list(range(1945, 2011))
anos_possiveis_str = ",".join(map(str, anos_possiveis))

df_nomes = df_nomes.withColumn("AnoNascimento", expr(f"element_at(array({anos_possiveis_str}), cast(rand() * {len(anos_possiveis)} as int) + 1)"))
df_nomes = df_nomes.withColumn("AnoNascimento", col("AnoNascimento").cast("int"))
df_nomes.printSchema()
df_nomes.show(10)

root
 |-- Nomes: string (nullable = true)
 |-- Escolaridade: string (nullable = false)
 |-- Pais: string (nullable = true)
 |-- AnoNascimento: integer (nullable = true)

+-----------------+------------+---------------+-------------+
|            Nomes|Escolaridade|           Pais|AnoNascimento|
+-----------------+------------+---------------+-------------+
|   Frances Bennet| Fundamental|       Suriname|         1967|
|    Jamie Russell|       Medio|Guiana Francesa|         1970|
|   Edward Kistler|    Superior|       Paraguai|         1997|
|    Sheila Maurer| Fundamental|       Suriname|         1991|
| Donald Golightly| Fundamental|        Uruguai|         1981|
|       David Gray|    Superior|        Equador|         1964|
|      Joy Bennett|    Superior|         Guiana|         2008|
|      Paul Kriese| Fundamental|        Equador|         1999|
|Berniece Ornellas|       Medio|      Argentina|         2002|
|    Brian Farrell|       Medio|       Colombia|         1994|
+----------

## Etapa 6

In [6]:
df_select = df_nomes.select("*").where(col("AnoNascimento") > 2000)
df_select.show(10)
count_filtered = df_select.count()

+-----------------+------------+---------+-------------+
|            Nomes|Escolaridade|     Pais|AnoNascimento|
+-----------------+------------+---------+-------------+
|      Joy Bennett|    Superior| Colombia|         2008|
|Berniece Ornellas|       Medio| Suriname|         2002|
|    Tracy Herring| Fundamental|  Equador|         2003|
|  Howard Lazarine| Fundamental|Venezuela|         2006|
|          Lois Ly|       Medio|Venezuela|         2007|
|       Daryl Page| Fundamental|Argentina|         2001|
|  Jerry Chynoweth|       Medio|Argentina|         2005|
|   Rosie Lovelady|       Medio| Suriname|         2001|
|      Donald Vogt|       Medio|    Chile|         2004|
|    George Miller|       Medio| Suriname|         2002|
+-----------------+------------+---------+-------------+
only showing top 10 rows



## Etapa 7

In [7]:
df_nomes.createOrReplaceTempView("nomes")
spark.sql("SELECT * FROM nomes WHERE AnoNascimento > 2000").show(10)

24/07/29 00:51:39 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------------+------------+---------------+-------------+
|            Nomes|Escolaridade|           Pais|AnoNascimento|
+-----------------+------------+---------------+-------------+
|      Joy Bennett|    Superior|      Argentina|         2008|
|Berniece Ornellas|       Medio|        Bolivia|         2002|
|    Tracy Herring| Fundamental|          Chile|         2003|
|  Howard Lazarine| Fundamental|       Colombia|         2006|
|          Lois Ly|       Medio|        Bolivia|         2007|
|       Daryl Page| Fundamental|        Equador|         2001|
|  Jerry Chynoweth|       Medio|           Peru|         2005|
|   Rosie Lovelady|       Medio|         Brasil|         2001|
|      Donald Vogt|       Medio|Guiana Francesa|         2004|
|    George Miller|       Medio|        Equador|         2002|
+-----------------+------------+---------------+-------------+
only showing top 10 rows



## Etapa 8

In [8]:
df_millenials = df_nomes.select("*").where((col("AnoNascimento") >= 1980) & (col("AnoNascimento") <= 1994))
df_millenials.show(10)

+----------------+------------+---------------+-------------+
|           Nomes|Escolaridade|           Pais|AnoNascimento|
+----------------+------------+---------------+-------------+
|   Sheila Maurer| Fundamental|       Suriname|         1991|
|Donald Golightly| Fundamental|      Argentina|         1981|
|   Brian Farrell|       Medio|        Bolivia|         1994|
|  Kara Mcelwaine| Fundamental|          Chile|         1990|
|    Ernest Hulet|    Superior|         Brasil|         1983|
|    David Medina|       Medio|          Chile|         1984|
|     Albert Leef|       Medio|       Paraguai|         1981|
|     Frank Wiley|    Superior|          Chile|         1983|
|Wallace Mitchell| Fundamental|Guiana Francesa|         1980|
|  Amanda Gravitt|    Superior|Guiana Francesa|         1983|
+----------------+------------+---------------+-------------+
only showing top 10 rows



## Etapa 9

In [9]:
df_nomes.createOrReplaceTempView("nomes")
spark.sql("SELECT * FROM nomes WHERE AnoNascimento >= 1980 AND AnoNascimento <= 1994").show(10)

+----------------+------------+---------+-------------+
|           Nomes|Escolaridade|     Pais|AnoNascimento|
+----------------+------------+---------+-------------+
|   Sheila Maurer| Fundamental|Argentina|         1991|
|Donald Golightly| Fundamental|     Peru|         1981|
|   Brian Farrell|       Medio| Colombia|         1994|
|  Kara Mcelwaine| Fundamental| Colombia|         1990|
|    Ernest Hulet|    Superior| Paraguai|         1983|
|    David Medina|       Medio|Argentina|         1984|
|     Albert Leef|       Medio|  Equador|         1981|
|     Frank Wiley|    Superior| Paraguai|         1983|
|Wallace Mitchell| Fundamental| Paraguai|         1980|
|  Amanda Gravitt|    Superior| Colombia|         1983|
+----------------+------------+---------+-------------+
only showing top 10 rows



## Etapa 10

In [10]:
spark.sql("""
SELECT 
Pais,
    CASE
        WHEN AnoNascimento BETWEEN 1944 AND 1964 THEN 'Baby Boomers'
        WHEN AnoNascimento BETWEEN 1965 AND 1979 THEN 'Geração X'
        WHEN AnoNascimento BETWEEN 1980 AND 1994 THEN 'Millennials'
        WHEN AnoNascimento BETWEEN 1995 AND 2015 THEN 'Geração Z'
    END AS Geracao,
    COUNT(*) AS Quantidade
FROM nomes
GROUP BY Pais, Geracao
ORDER BY Pais, Geracao, Quantidade
""").show()

+---------+------------+----------+
|     Pais|     Geracao|Quantidade|
+---------+------------+----------+
|Argentina|Baby Boomers|    233464|
|Argentina|   Geração X|    174618|
|Argentina|   Geração Z|    186272|
|Argentina| Millennials|    175020|
|  Bolivia|Baby Boomers|    233901|
|  Bolivia|   Geração X|    174587|
|  Bolivia|   Geração Z|    186539|
|  Bolivia| Millennials|    174572|
|   Brasil|Baby Boomers|    233333|
|   Brasil|   Geração X|    175459|
|   Brasil|   Geração Z|    187081|
|   Brasil| Millennials|    174470|
|    Chile|Baby Boomers|    233545|
|    Chile|   Geração X|    174970|
|    Chile|   Geração Z|    186742|
|    Chile| Millennials|    174722|
| Colombia|Baby Boomers|    232799|
| Colombia|   Geração X|    174378|
| Colombia|   Geração Z|    186671|
| Colombia| Millennials|    175059|
+---------+------------+----------+
only showing top 20 rows



In [11]:
spark.stop()